In [2]:
from google.colab import drive
import pandas as pd

drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
comments_012 = pd.read_csv("/content/drive/MyDrive/THESIS_May15/comments_012_may5.csv")
comments_345 = pd.read_csv("/content/drive/MyDrive/THESIS_May15/comments_345_may5.csv")
comments_67 = pd.read_csv("/content/drive/MyDrive/THESIS_May15/comments_67_may5.csv")
comments_89 = pd.read_csv("/content/drive/MyDrive/THESIS_May15/comments_89_may5.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(comments_012.columns)
print(comments_345.columns)
print(comments_67.columns)
print(comments_89.columns)

Index(['Unnamed: 0', '0'], dtype='object')
Index(['Unnamed: 0', '0'], dtype='object')
Index(['Unnamed: 0', '0'], dtype='object')
Index(['Unnamed: 0', '0'], dtype='object')


In [5]:
comments_012 = comments_012['0']
comments_345 = comments_345['0']
comments_67 = comments_67['0']
comments_89 = comments_89['0']

In [6]:
comments_all = pd.concat([comments_012, comments_345, comments_67, comments_89], ignore_index = True)
comments_all.shape

(17073732,)

In [7]:
comments_all.head()

0    body\nSS:  Jade Helm is welcomed now.  But wha...
1    At first, when I heard about the march, I thou...
2    \nLooks that way to me. Watch the speech he ga...
3    Sincerely from an american-- this country is l...
4    \nThis sub would an apocalyptic wasteland if t...
Name: 0, dtype: object

In [8]:
comments_all = comments_all[comments_all != "[deleted]"]
comments_all = comments_all[comments_all != "[removed]"]
comments_all.shape

(17073731,)

In [9]:
comments_all.dropna(inplace = True)

In [10]:
comments_all.drop_duplicates(inplace = True)

In [11]:
comments_all = comments_all.astype("str")

POST LENGTH DISCUSSIONS BEFORE DATA CLEANING

In [12]:
post_length = comments_all.apply(lambda x: len(x))

In [13]:
type (post_length)

pandas.core.series.Series

In [14]:
comments_df = pd.concat([comments_all, post_length], axis = 1, join = "inner")

comments_df.head()

,0,0
0,body\nSS: Jade Helm is welcomed now. But wha...,152
1,"At first, when I heard about the march, I thou...",421
2,\nLooks that way to me. Watch the speech he ga...,90
3,Sincerely from an american-- this country is l...,659
4,\nThis sub would an apocalyptic wasteland if t...,99


In [15]:
comments_df.columns = ["comments", "length"]

In [16]:
comments_chosen = comments_df.loc[comments_df["length"] >= 482, "comments"] #taking the value 482 because it was the median value for post length before posts were cleaned

In [17]:
comments_chosen.shape

(1350485,)

In [18]:
comments_chosen = comments_chosen.astype("str")

In [19]:
import re

def clean_text_comment_vrs(given_text):
  
  message = re.sub(r'(http[s]?\S+)', ' ', given_text) #Replace URLs with space because it might be too freq in this class   
  message = re.sub(r'(w{2}\.\S+)', ' ', message) #Replace URLs with space because it might be too freq in this class    
  message = re.sub(r'&[a-zA-Z]*', ' ', message) #also another code to remove the stuff with ampersand    
  message = re.sub(r'[^\w\d\s]', ' ', message) #punctuation removed (it removes * too!)
  message = re.sub(r'\d+(\.\d+)?', ' ', message)  #Replace numbers with space   
  message = re.sub(r'\s+', ' ', message) #too muchs space removed
  message = re.sub(r'^\s+|\s+?$', '', message.lower()) #Get rid of spaces at the beginning and at the end

  return message

In [20]:
cleaned_comments = comments_chosen.apply(clean_text_comment_vrs) #wait time around 3 mins

In [21]:
cleaned_comments = cleaned_comments[cleaned_comments != r""]

In [22]:
cleaned_comments.drop_duplicates(inplace = True)
cleaned_comments.shape

(1342090,)

In [23]:
cleaned_comments.dropna(inplace = True)
cleaned_comments.shape

(1342090,)

In [24]:
cleaned_comments_list = cleaned_comments.tolist()
first_comment = cleaned_comments_list[0]

In [25]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
from nltk import word_tokenize

In [33]:
comments_df_str = comments_df.astype("str")

comments_df_str.head()

,comments,length
0,body\nSS: Jade Helm is welcomed now. But wha...,152
1,"At first, when I heard about the march, I thou...",421
2,\nLooks that way to me. Watch the speech he ga...,90
3,Sincerely from an american-- this country is l...,659
4,\nThis sub would an apocalyptic wasteland if t...,99


In [ ]:
comment_trial = comments_df_str.iloc[0, :]

cleaned_comment = comment_trial.apply(clean_text_comment_vrs)
cleaned_comment = cleaned_comment.astype("str")

tokens = word_tokenize(cleaned_comment[0])
tokens

In [57]:
import os
os.chdir("/content/drive/MyDrive/THESIS_May15")

cleaned_comments.to_csv("cleaned_comments_482threshold.csv")